In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
import time
import numpy as np
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth', 20)

In [2]:
def get_details(driver): 

    # Getting attraction details
    pane = driver.find_element_by_id("pane")

    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button[class^='CsEnBe']")))

    time.sleep(2)

    try:
  
        elements = pane.find_elements_by_css_selector("div[class^='QSFF4-text gm2-body-2']")
        for i in range(len(elements)):
            if i  == 0:
                address = elements[i].text
            elif '+' in elements[i].text:
                plus_code = elements[i].text
    
    except:
        address = np.nan
        plus_code = np.nan

    elements2 = driver.find_elements_by_class_name("h0ySl-wcwwM-E70qVe")
    for i in range(len(elements)):
        if i == 1:
            category = elements2[1].text
    review_rating = driver.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d").text
       
    # attraction_details = {'address': address, 'plus code': plus_code, "category": category, "review rating": review_rating}

    return [address, plus_code, category, float(review_rating)]

In [3]:
def get_keywords(driver):

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "widget-pane-link")))

    driver.find_element_by_class_name("widget-pane-link").click() # click on review page

    time.sleep(2)

    wait = WebDriverWait(driver,20)
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id='pane']/div/div[1]/div/div/div[2]/div[7]/div[2]/div/button")))
    
    # # Sort by Newest Reviews
    # driver.find_element_by_xpath("//*[@id='pane']/div/div[1]/div/div/div[2]/div[7]/div[2]/div/button").click()
    # driver.find_element_by_xpath("//*[@id='action-menu']/ul/li[2]").click()  # select newest option

    # Getting reviews based on keywords
    reviews_panel = driver.find_element_by_css_selector("div[class^='widget-pane widget-pane-visible']")
    filter_kw = reviews_panel.find_elements_by_css_selector("button[class^= 'tXNTee gm2-hairline-border rmF2qb-YPqjbf-XPtOyb-LgbsSe NIyLF-haAclf']")

    keywords = []
    for i in filter_kw:
        if i.text != 'All' and len(i.text) != 0:
            kw = i.text.split()
            keywords.append(kw[0])
        
    return keywords   
    

In [4]:
attr_list = pd.read_csv("./Data/Tourist Attractions/Tourist Attractions List.csv")
attr_list.head()

,name,category,sub_category,latitude,longtitude,description,address,x_addr,y_addr,url
0,Chinatown Herita...,culture heritage,heritage discovery,1.283510,103.844350,Experience how S...,48 Pagoda Street,29227.7061,29549.5371,http://www.singa...
1,Thian Hock Keng ...,culture heritage,places of worship,1.280940,103.847630,Beautifully rest...,158 Telok Ayer S...,29592.7445,29265.3615,http://www.thian...
2,Eurasian Heritag...,culture heritage,heritage discovery,1.310070,103.899420,This engaging he...,139 Ceylon Road,35356.4676,32486.4992,http://www.euras...
3,Shophouses,architecture,historical,1.277219,103.837336,With many beauti...,NaN,28447.0623,28853.8642,NaN
4,Pinnacle@Duxton,architecture,modern,1.275490,103.841420,The soaring line...,1 Cantonment Road,28901.6240,28662.7257,http://www.pinna...


In [5]:
driver = webdriver.Chrome(executable_path = "C:/Users/Gi Han/OneDrive - Singapore Management University/Documents/Y3T1/chromedriver_win32/chromedriver.exe")

attractions = attr_list['name'] 

all_att = {}
failed_details = []

for att in attractions:

    try:
    
        # Entering the attraction name

        driver.get("https://www.google.com.sg/maps/@1.3893632,103.7427559,15z?hl=en")

        search_box = driver.find_element_by_id("searchboxinput")
        search_box.send_keys(att)

        driver.find_element_by_id('searchbox-searchbutton').click()

        time.sleep(5)
        
        url_details = driver.current_url
        
        attraction_details = get_details(driver)
        # address, plus_code, category, review_rating
        print(att + ': ' + str(attraction_details))
        
        if "name" not in all_att:
            all_att['name'] = [att]
        else:
            all_att['name'].append(att)

        if "address" not in all_att:
            all_att["address"] = [attraction_details[0]]
        else:
            all_att["address"].append(attraction_details[0])

        if "plus_code" not in all_att:
            all_att["plus_code"] = [attraction_details[1]]
        else:
            all_att["plus_code"].append(attraction_details[1])
        
        if "category" not in all_att:
            all_att["category"] = [attraction_details[2]]
        else:
            all_att["category"].append(attraction_details[2])

        if "overall_rating" not in all_att:
            all_att["overall_rating"] = [attraction_details[3]]
        else:
            all_att["overall_rating"].append(attraction_details[3])
        
        if "url" not in all_att:
            all_att["url"] = [url_details]
        else:
            all_att["url"].append(url_details)

    except:

        failed_details.append(att)
        print('*' * 40)
        print('FAILED: ', att)
        print('*' * 40)
        continue

    try:
    
        keywords = get_keywords(driver)

        if "keywords" not in all_att:
            all_att["keywords"] = [keywords]
        else:
            all_att["keywords"].append(keywords)

        print('kw: ', keywords)
    
    except:

        if "keywords" not in all_att:
            all_att["keywords"] = [np.NaN]
        else:
            all_att["keywords"].append(np.NaN)

        print('-' * 40)
        print('FAILED (kw): ', att)
        print('-' * 40)
        

Chinatown Heritage Centre, Singapore: ['48 Pagoda St, Singapore 059207', '7RMV+9Q Singapore', 'Ethnographic museum', 4.4]
kw:  ['souvenirs', 'immigrants', 'audio', 'life', 'market', 'street', 'chinese', 'chinese', '1950s', 'settlers']
Thian Hock Keng Temple, Singapore: ['158 Telok Ayer St, Singapore 068613', '7RJX+C3 Singapore', 'Taoist temple', 4.4]
kw:  ['history', 'chinese', 'peaceful', 'sea', 'architecture', 'goddess', 'heritage', 'wish', 'confucius', '1973']
Eurasian Heritage Centre: Singapore Attraction: ['139 Ceylon Rd, The Eurasian Association, Singapore 429744', '8V6X+2Q Singapore', 'Heritage museum', 4.7]
kw:  []
****************************************
FAILED:  Shophouses
****************************************
Pinnacle@Duxton: ['1G Cantonment Rd, Singapore 085301', '7RGR+QF Singapore', 'Housing complex', 4.4]
kw:  ['floor', 'pay', 'garden', 'fee', 'public', 'rooftop', 'lift', 'architecture', 'office', 'island']
Hajjah Fatimah Mosque: ['4001 Beach Rd, Singapore 199584', '8V

In [9]:
print(len(failed_details))
str(failed_details)

31


'[\'Shophouses\', \'Capitol Building Singapore\', \'The Interlace\', \'Gillman Barracks – YourSingapore\', \'Raffles Statue – Sir Stamford Raffles Landing Site\', \'Reflections at Bukit Chandu: Singapore War Memorial\', \'The Civilian War Memorial park in Singapore\', \'Sri Veeramakaliamman Temple: Hindu Temple in Singapore\', \'Contemporary Arts Center\', \'Singapore Art Museum\', \'Pulau Ubin & Singapore Islands\', \'Little India Singapore: History & Culture\', "Orchard Road, Singapore: Asia\'s Most Famous Shopping Street", \'MacRitchie Singapore & Singapore Nature Reserve\', \'Civic District, Singapore\', \'HarbourFront, Singapore\', \'Marina Bay Sands®\', \'Marina Bay Singapore: Attractions & Things to do\', \'Marina Bay Sands, Singapore\', \'Merlion Park, Singapore: Attractions & Things to Do\', \'Marina Bay Sands® SkyPark\', \'Resorts World Sentosa, Singapore\', \'Singapore Garden & Singapore Park\', \'Sungei Buloh\', \'Singapore River: History & Culture Tours\', \'Joo Chiat/Kato

## Converting to csv

In [7]:
all_df = pd.DataFrame(all_att)
all_df

,name,address,plus_code,category,overall_rating,url,keywords
0,Chinatown Herita...,"48 Pagoda St, Si...",7RMV+9Q Singapore,Ethnographic museum,4.4,https://www.goog...,"[souvenirs, immi..."
1,Thian Hock Keng ...,158 Telok Ayer S...,7RJX+C3 Singapore,Taoist temple,4.4,https://www.goog...,"[history, chines..."
2,Eurasian Heritag...,"139 Ceylon Rd, T...",8V6X+2Q Singapore,Heritage museum,4.7,https://www.goog...,[]
3,Pinnacle@Duxton,1G Cantonment Rd...,7RGR+QF Singapore,Housing complex,4.4,https://www.goog...,"[floor, pay, gar..."
4,Hajjah Fatimah M...,"4001 Beach Rd, S...",8V37+54 Singapore,Mosque,4.7,https://www.goog...,"[peaceful, beaut..."
...,...,...,...,...,...,...,...
70,East Coast Park ...,E Coast Park Ser...,8W26+8V Singapore,Park,4.5,https://www.goog...,"[beach, bike, se..."
71,Fort Canning Park,"River Valley Rd,...",7RVW+GM Singapore,Park,4.5,https://www.goog...,"[history, exhibi..."
72,Jurong Bird Park...,"2 Jurong Hill, 6...",8P94+FH Singapore,Wildlife park,4.5,https://www.goog...,"[shows, penguins..."
73,Adventure Cove W...,8 Sentosa Gatewa...,7R59+GF Singapore,Water park,4.4,https://www.goog...,"[snorkeling, que..."


In [8]:
all_df.to_csv("./Data/Google Review Raw Data/Attractions_url.csv", index=False)

## Attractions with 4 stars and above

In [35]:
selected_df = all_df[all_df['overall_rating'] >= 4.0]
selected_df

,name,address,plus_code,category,overall_rating,url,keywords
0,Chinatown Herita...,"48 Pagoda St, Si...",7RMV+9Q Singapore,Ethnographic museum,4.4,https://www.goog...,"[souvenirs, immi..."
1,Thian Hock Keng ...,158 Telok Ayer S...,7RJX+C3 Singapore,Taoist temple,4.4,https://www.goog...,"[history, chines..."
2,Eurasian Heritag...,"139 Ceylon Rd, T...",8V6X+2Q Singapore,Heritage museum,4.7,https://www.goog...,[]
3,Pinnacle@Duxton,1G Cantonment Rd...,7RGR+QF Singapore,Housing complex,4.4,https://www.goog...,"[floor, pay, gar..."
4,Hajjah Fatimah M...,"4001 Beach Rd, S...",8V37+54 Singapore,Mosque,4.7,https://www.goog...,"[peaceful, beaut..."
...,...,...,...,...,...,...,...
70,East Coast Park ...,E Coast Park Ser...,8W26+8V Singapore,Park,4.5,https://www.goog...,"[beach, bike, se..."
71,Fort Canning Park,"River Valley Rd,...",7RVW+GM Singapore,Park,4.5,https://www.goog...,"[history, exhibi..."
72,Jurong Bird Park...,"2 Jurong Hill, 6...",8P94+FH Singapore,Wildlife park,4.5,https://www.goog...,"[shows, penguins..."
73,Adventure Cove W...,8 Sentosa Gatewa...,7R59+GF Singapore,Water park,4.4,https://www.goog...,"[snorkeling, que..."


In [12]:
selected_df.to_csv("./Data/Google Review Raw Data/Attractions(selected)_url.csv", index=False)

## 2nd round of scraping keywords 
(for those that failed scraping of details)

In [13]:
kw_df = pd.read_csv('./Data/Google Review Raw Data/attractions_scrape.csv')
kw_df.head()

,name,address,plus_code,category,overall_rating,url
0,Capitol Building...,"17 Stamford Rd, ...",7VV2+9G Singapore,Performing arts ...,4.4,https://www.goog...
1,Gillman Barracks...,"9 Lock Rd, Singa...",7RH3+8Q Singapore,Arts organization,4.3,https://www.goog...
2,Raffles Statue �...,"North Bank, Sing...",7VQ2+38 Singapore,Tourist attraction,4.4,https://www.goog...
3,Reflections at B...,31 - K Pepys Roa...,7QHV+VM Singapore,War museum,4.1,https://www.goog...
4,Sri Veeramakalia...,141 Serangoon Rd...,8V52+6V Singapore,Hindu temple,4.6,https://www.goog...


In [24]:
driver = webdriver.Chrome(executable_path = "C:/Users/Gi Han/OneDrive - Singapore Management University/Documents/Y3T1/chromedriver_win32/chromedriver.exe")

keywords_list = []
failed_kw = []

for index, row in kw_df.iterrows():

    driver.get(row['url'])

    try:
        kw = get_keywords(driver)
        keywords_list.append(kw)
    
    except:
        keywords_list.append(np.nan)
        failed_kw.append(row['name'])
        print('*' * 40)
        print('FAILED: ', row['name'])
        print('*' * 40)   


In [25]:
kw_df['keywords'] = keywords_list
kw_df.head()

,name,address,plus_code,category,overall_rating,url,keywords
0,Capitol Building...,"17 Stamford Rd, ...",7VV2+9G Singapore,Performing arts ...,4.4,https://www.goog...,"[building, nosta..."
1,Gillman Barracks...,"9 Lock Rd, Singa...",7RH3+8Q Singapore,Arts organization,4.3,https://www.goog...,"[galleries, rest..."
2,Raffles Statue �...,"North Bank, Sing...",7VQ2+38 Singapore,Tourist attraction,4.4,https://www.goog...,"[history, good, ..."
3,Reflections at B...,31 - K Pepys Roa...,7QHV+VM Singapore,War museum,4.1,https://www.goog...,"[WWII, learn, wa..."
4,Sri Veeramakalia...,141 Serangoon Rd...,8V52+6V Singapore,Hindu temple,4.6,https://www.goog...,"[pray, tourists,..."


In [26]:
print(len(failed_kw))
failed_kw

0


[]

In [36]:
final_df = pd.concat([selected_df, kw_df])
final_df.tail()

,name,address,plus_code,category,overall_rating,url,keywords
19,Joo Chiat Rd,NaN,NaN,Notable street,4.8,https://www.goog...,[houses]
20,"Sentosa Island, ...",NaN,NaN,Island,4.6,https://www.goog...,"[entertainment, ..."
21,Universal Studio...,8 Sentosa Gatewa...,7R3F+JG Singapore,Theme park,4.6,https://www.goog...,"[queue, jurassic..."
22,Kusu Island,NaN,NaN,Island,4.3,https://www.goog...,"[good, picnic, s..."
23,HortPark,"33 Hyderabad Rd,...",7QHX+JX Singapore,Park,4.4,https://www.goog...,"[flowers, weddin..."


In [37]:
final_df.to_csv("./Data/Google Review Raw Data/Attractions_url (final).csv", index=False)